# SECK Mouhamadou Abdoulaye
# abdouseck.tiv@gmail.com

## Deep Learning et réseau RNN (Recurent Neural Network)

L'objectif de cette manipulation est de vous montrer le potentiel du deep learning en NLP et plus particulirerment en NLG (Natural Language Generation)


#### Définition
Les réseaux de neurones récurrents, sont une classe de réseaux de neurones qui permettent aux prédictions antérieures d'être utilisées comme entrées, par le biais d'états cachés (en anglais hidden states)
Un RNN a la particularité d'avoir une memoire sur les bonnes predictions de la sequence de sortie (en fonction de la sequence d'entrée).
Les modèles RNN sont surtout utilisés dans les domaines du traitement automatique du langage naturel et de la reconnaissance vocale.


![Title](https://www.i2tutorials.com/wp-content/media/2019/09/Neural-network-62-i2tutorials.png)


![Title](https://miro.medium.com/max/600/1*pQ2tm6Mirdrf6hqwfYXb0g.gif)

#### Variantes aux RNNs traditionnels,
Les unités de porte récurrente (en anglais Gated Recurrent Unit) (GRU) et les unités de mémoire à long/court terme (en anglais Long Short-Term Memory units) (LSTM) où le LSTM peut être vu comme étant une généralisation du GRU. Ils apaisent le problème du "gradient qui disparait", rencontré par les RNNs traditionnels



#### Sequence to Sequence 

La séquence à séquence (Seq2Seq) consiste à entraîner des modèles pour convertir des séquences d'un domaine (par exemple des phrases en anglais) en séquences d'un autre domaine (par exemple les mêmes phrases traduites en français).

Cela peut être utilisé pour la traduction automatique ou pour la réponse aux questions sans réponse (générant une réponse en langage naturel à partir d'une question en langage naturel) - en général, il est applicable chaque fois que vous avez besoin de générer du texte.

Il existe plusieurs façons de gérer cette tâche, dont celle utilisant des RNN.



Dans le cas général, les séquences d'entrée et les séquences de sortie ont des longueurs différentes (par exemple, traduction automatique) et la séquence d'entrée entière est nécessaire pour commencer à prédire la cible. Cela nécessite une configuration plus avancée, ce à quoi les gens se réfèrent généralement lorsqu'ils mentionnent des «modèles de séquence à séquence» sans autre contexte. Voilà comment cela fonctionne:

+ Une couche RNN (ou plusieurs couches) joue le rôle d'"encodeur": elle traite la séquence d'entrée et renvoie son propre état interne. Notez que nous rejetons les sorties de l'encodeur RNN, ne **récupérant que l'état**. Cet état servira de "contexte", ou "conditionnement", du décodeur à l'étape suivante.

+ Une autre couche RNN (ou plusieurs couches) fait office de "décodeur": elle est entraînée pour prédire les caractères suivants de la séquence cible, étant donné les caractères précédents de la séquence cible. Plus précisément, il est formé pour transformer les séquences cibles en décalées d'un pas de temps dans le futur. Il est important de noter que l'encodeur utilise comme état initial les vecteurs d'état de l'encodeur, ce qui permet au décodeur d'obtenir des informations sur ce qu'il est censé générer. En effet, le décodeur apprend à générer des cibles [t + 1 ...] en fonction de [... t], conditionnées à la séquence d'entrée (hidden state).

Nous allons aborder les model de type sequence to sequence et en particulier le Long Short Term Memory (LSTM) tres utilisé egalement en Time series.

![ee](https://metalblog.ctif.com/wp-content/uploads/sites/3/2021/09/Les-differents-types-de-reseaux-de-neurones-RNN-LSTM-et-GRU-1024x307.jpg)

![Title](https://upload.wikimedia.org/wikipedia/commons/6/63/Long_Short-Term_Memory.svg)


https://penseeartificielle.fr/comprendre-lstm-gru-fonctionnement-schema/

L'approche est traditionnelle: 

- Charger une corpus d'entrainement
- Filtrer ce texte
- Nettoyer intelligemment le texte
- Tokenizer les mots puis les textes (qui seront alors un sequence de mots)
- Normaliser la longueur des textes donnés en entrée du modele
- Créer le modele de LSTM
- Entrainer le modele
- Generer du texte

#### Charger une corpus d'entrainement

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Utilisons keras  pour construire notre Réseau
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku 

from numpy.random import seed

import pandas as pd
import numpy as np
import string, os 


import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
import pandas as pd

df=pd.read_excel('LOR_3.xlsx')
text='. '.join(df['Commentaire'])

In [3]:
df.head()

,Data,Score,Commentaire
0,11 juin 2013,"5,0",Commentaire valable pour l'ensemble de la tril...
1,1 mai 2013,"5,0","3h fantastiques, avec scènes de bataille épiqu..."
2,3 juin 2013,"5,0",Si tu as aimé La Communauté de l'Anneau et Les...
3,11 novembre 2013,"5,0","""Le retour du roi"" clôture la trilogie du Seig..."
4,14 octobre 2013,"5,0",L'épisode final de la légendaire sage du Seign...


In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
# Pour la bible 
with open('la_bible_nouveau_testament.txt','r',encoding='utf8') as f:
    text=f.read()
    

#### Nettoyer intelligemment le texte

In [6]:

import re

stop=[]


def tok_me(texte):
    # J'enleve la ponctuation et je mets en minuscule
    #p="([aA-zZéèàùîêâûôçïëœ0-9]{1,})"
    p="([aA-zZéèàùîêâûôçïëœ.,:!?;]{1,})"
    texte=' '.join(re.findall(p,texte))
    #Je renvoi une liste de token
    return texte.lower().split()

def stop_me(liste_token):
    final=[]
    for token in liste_token:
        if token in stop:
            continue
        final.append(token)
    return final

def preprocess(texte):
    return ' '.join(stop_me(tok_me(texte)))

#Corpus=df['Commentaire']

texte=[elem for elem in re.split('!|\?|\.{1,}| \-{1,}|;|:', text.replace('.',' . ').replace(',',' , ').replace(':',' : ').replace('?',' ? ')) if (len(str(elem).split()) < 100)]

com=list(map(preprocess,texte))


# On prend les phrases avec + 2 mots: 
com = [elem for elem in com if len(elem.split()) > 2]


In [7]:
com[7],len(texte)

('et aram engendra aminadab', 19030)

#### Tokenizer les mots puis les textes

Le but de cette modelisation est de prédire le/les mots suivant une sequence (mots/phrases) donnée en entrée.

Dans la fonction suivante, nous allons utiliser le Tokenizer de keras pour extraire les elements uniques (mot) de chaque phrase puis leur donner un id.

Nous allons, dans inp_sequences, ecrire toutes les combinaisons (n_gram) de token pour chaque phrase.

ex : "Je vais bien merci beaucoup"

[Je,vais]
[Je,vais,bien]
[Je vais bien merci] 
etc...

C'est comme cela que nous preparons l'entrainement : telle sequence d'id doit me donner le suivant, etc ...


In [8]:
import random

tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    #Tokenizer : associe à chaque mot unique un ID 
    
    ## Text to sequence of tokens (mots uniques) 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(random.sample(com, 3600))
len(inp_sequences),inp_sequences[:10]

(46755,
 [[112, 23],
  [112, 23, 9],
  [112, 23, 9, 8],
  [112, 23, 9, 8, 54],
  [112, 23, 9, 8, 54, 9],
  [112, 23, 9, 8, 54, 9, 8],
  [112, 23, 9, 8, 54, 9, 8, 193],
  [112, 23, 9, 8, 54, 9, 8, 193, 1593],
  [112, 23, 9, 8, 54, 9, 8, 193, 1593, 204],
  [112, 23, 9, 8, 54, 9, 8, 193, 1593, 204, 8]])

#### Nous pouvons traduire ces sequences

In [9]:
print(list(map((lambda x: [tokenizer.index_word[elem] for elem in x]), inp_sequences[:10])))

[['non', 'mais'], ['non', 'mais', 'que'], ['non', 'mais', 'que', 'les'], ['non', 'mais', 'que', 'les', 'choses'], ['non', 'mais', 'que', 'les', 'choses', 'que'], ['non', 'mais', 'que', 'les', 'choses', 'que', 'les'], ['non', 'mais', 'que', 'les', 'choses', 'que', 'les', 'nations'], ['non', 'mais', 'que', 'les', 'choses', 'que', 'les', 'nations', 'sacrifient'], ['non', 'mais', 'que', 'les', 'choses', 'que', 'les', 'nations', 'sacrifient', 'elles'], ['non', 'mais', 'que', 'les', 'choses', 'que', 'les', 'nations', 'sacrifient', 'elles', 'les']]


#### Normaliser la longueur des textes donnés en entrée du modele

Dans la cellule suivante, nous creons l'echantillon de train sur la base des in_sequence generées plus haut.

La subtilité, c'est de pouvoir livrer à la machine une donnée d'entrée toujours de meme taille/format


In [10]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [11]:
i=15
print("L'input",predictors[i],tokenizer.index_word[predictors[i].argmax()])
print("L'output, c'est à dire :",label[i].argmax()," soit :", tokenizer.index_word[label[i].argmax()])

L'input [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0  112   23    9    8   54
    9    8  193 1593  204    8 1593    6   19  483    1] tout
L'output, c'est à dire : 112  soit : non


La sortie est un vecteur one hot encoder

#### Créer le modele de LSTM

Nous avons ainsi nos trains (predictor) et le target associé (label)

Input Layer : Sequence de mots

Petite couche d'embedding 

LSTM Layer : Determine la sortie via les cellules de LSTM 

Dropout Layer :Une reductions du nombre de cellules (regularisation) qui va aleatoirement eteindre les neurones LSTM instables ==> Eviter l'overfitting 

Output Layer : Calcule une probabilité (softmax !) du meilleur choix en sortie 


![title](http://www.shivambansal.com/blog/text-lstm/2.png)

#### Subtilité Bidirectionnal LSTM

Le modèle prédit un mot en fonction à la fois des termes qui le précèdent et  de ceux qui lui succèdent. La fonction de coût est une simple moyenne des fonctions de coût calculées par les deux parties du bi-LSTM.

In [12]:
total_words,max_sequence_len

(5277, 82)

In [17]:
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.optimizers import SGD,Adamax,Adagrad,RMSprop


def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Input Embedding Layer : Transforme les entiers positifs (index) en vecteurs de float de taille fixe.
    # La sequence d'input ([0 0 0 id3 id2 id1]) ==> ([Emb(0 Emb(0) ... Emb(id3) Emb(id2 Emb(id1))])
    model.add(Embedding(total_words, 20, input_length=input_len))
    print(model.output_shape)
    
    # LSTM Layer nous retournons la sequence entiere à la couche LSTM suivante qui en a besoin
    #model.add(Bidirectional(LSTM(48, dropout=0.1, return_sequences=True), input_shape=(input_len, 1)))
    #model.add(LSTM(64,return_sequences=True,dropout=0.1))
    # LSTM Layer 
    #model.add(LSTM(64,return_sequences=False,dropout=0.15))
    model.add(Bidirectional(LSTM(48, return_sequences=False,dropout=0.1), input_shape=(input_len, 1)))
    
    #model.add(Dense(model.output_shape[-1], activation='tanh'))
    # Output Layer recherche du meilleur candidat sur un vecteur de la taille du vocabulaire
    # Vecteur de sortie ([score1,score2,score3,...., score_nb_word_vocab].argmax ==> id ayant le score max)
    model.add(Dense(total_words, activation='softmax'))
    
    # wi(i+1) = wi (i) - alpha x (d E / d wi)
    opt=Adamax(learning_rate=0.09)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer=opt)
    print(model.output_shape)
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

(None, 81, 20)
(None, 5277)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 81, 20)            105540    
                                                                 
 bidirectional_1 (Bidirectio  (None, 96)               26496     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 5277)              511869    
                                                                 
Total params: 643,905
Trainable params: 643,905
Non-trainable params: 0
_________________________________________________________________


#### Entrainer le modele


In [23]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)


history = model.fit(predictors, label,
    epochs=20, 
    batch_size=50,validation_split=0.2,callbacks=[callback]
)

Epoch 1/20
749/749 [==============================] - 73s 97ms/step - loss: 4.6834 - accuracy: 0.1683 - val_loss: 5.6536 - val_accuracy: 0.1555
Epoch 2/20
749/749 [==============================] - 74s 99ms/step - loss: 4.4279 - accuracy: 0.1804 - val_loss: 5.7104 - val_accuracy: 0.1562
Epoch 3/20
749/749 [==============================] - 74s 99ms/step - loss: 4.2054 - accuracy: 0.1922 - val_loss: 5.8330 - val_accuracy: 0.1546
Epoch 4/20
749/749 [==============================] - 74s 99ms/step - loss: 4.0427 - accuracy: 0.2025 - val_loss: 5.9941 - val_accuracy: 0.1575
Epoch 5/20
749/749 [==============================] - 74s 99ms/step - loss: 3.8958 - accuracy: 0.2115 - val_loss: 6.1294 - val_accuracy: 0.1448


#### Générer du texte

Enfin, pour la prediction, une fonction qui tokenize le debut du texte, qui retrouve les ids mots associés, qui predit le mot le plus probable

In [25]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for i in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        #predicted = model.predict_classes(token_list, verbose=0)
        predict_x=model.predict(token_list) 
        predicted=np.argmax(predict_x,axis=1)
        #print("l'index du score max du on hot encoding de sortie du modele", predicted)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text

un exemple d'appel à cette fonction prediction

In [26]:
import warnings
warnings.filterwarnings('ignore')
print('##########################################')
print (generate_text(preprocess("Tout est incroyable, du début à la fin. J'ai adoré"), 8, model, max_sequence_len))
print (generate_text(preprocess("Ce film est vraiment le meilleur de la trilogie"), 8, model, max_sequence_len))
print (generate_text(preprocess("Le seigneur des anneaux est une longue histoire, ce dernier épisode est toujours"), 8, model, max_sequence_len))
print (generate_text(preprocess("J'ai vraiment détesté, ce dernier"), 8, model, max_sequence_len))


##########################################
1/1 [==============================] - 0s 47ms/step
tout est incroyable, du début à la fin. j ai adoré fait connaître à la droite de dieu et
1/1 [==============================] - 0s 39ms/step
ce film est vraiment le meilleur de la trilogie gloire et que nous avons trouvé l assemblée
1/1 [==============================] - 0s 48ms/step
le seigneur des anneaux est une longue histoire, ce dernier épisode est toujours et de la gloire et que nous avons
1/1 [==============================] - 0s 39ms/step
j ai vraiment détesté, ce dernier en moi amène un corps ne sont pas
